---
jupyter: python3
title: Getting to know the data
---

In [121]:
import polars as pl
import numpy as np
from IPython.display import Markdown
from tabulate import tabulate

np.random.seed(1)

Let's start by reading in the data and printing some five random rows of it to get a sense for what we're dealing with.

In [122]:
# | column: page

df = pl.read_csv("../data/data.csv", separator=";", try_parse_dates=True)
df.sample(n=5)

Rank,Mark,Competitor,DOB,Nat,Pos,Venue,Date,Results Score,Mark [meters or seconds],Event,Wind,Sex
i64,str,str,date,str,str,str,date,i64,f64,str,str,str
8277,"""20.55""","""Rodney MARTIN""",1982-12-22,"""USA""","""1""","""Kalamata (GRE)…",2006-06-03,1135,20.5,"""200 Metres""","""1.2""","""male"""
10731,"""10.2""","""Stanley FLOYD""",1961-06-23,"""USA""","""2""","""Köln (GER)""",1981-08-23,1138,10.0,"""100 Metres""","""0.4""","""male"""
6152,"""76.63""","""Aleksey ZAGORN…",1978-05-31,"""RUS""","""1""","""Adler, Sochi (…",2010-03-04,1150,76.6,"""Hammer Throw""",null,"""male"""
99,"""4.83""","""Yelena ISINBAY…",1982-06-03,"""RUS""","""1""","""Olympiastadion…",2009-06-14,1217,4.8,"""Pole Vault""",null,"""female"""
1260,"""27:39.12""","""Brahim BOUTAYE…",1967-08-15,"""MAR""","""5""","""Oslo (NOR)""",1988-07-02,1164,1659.1,"""10000 Metres""",null,"""male"""


We can easily understand the different columns:

- `Rank` is the rank of that performance for the given sex and event.
- The `Mark` is the unparsed result entry for that performance. This can be in minutes, seconds, hours, meters (long jump) or points (decathlon).
- `Competitor`, `DOB`, and `Nat` are the competitor's name, date of birth, and nationality.
- `Pos` is the position that was achieved with this performance in that specific event where it was performed.
- `Venue` and `Date` specify where and when the performance was achieved.
- World Athletics assigns a score to a performance, that's what `Results Score` is.
- `Mark [meters or seconds]` is my attempt to parse the `Mark` into seconds or meters, i.e. a `float`. 
- `Event` is the event name.
- `Wind`, if available, tells the wind reading for that performance. This is mostly important for sprinting and jumping.
- `Sex` is either female or male.

## Some basic counts

Below are some basic counts of the data.

In [123]:
print("Shape of the dataframe:")
df.shape

Shape of the dataframe:


(463847, 13)

In [124]:
print("Counts for male and female performance:")
df.groupby("Sex").count()

Counts for male and female performance:


Sex,count
str,u32
"""male""",236187
"""female""",227660


In [125]:
print("Performance count by sex and event, colored:")
(
    df.groupby("Sex", "Event")
    .count()
    .pivot(index="Event", columns="Sex", values="count", aggregate_function=None)
    .sort("female", descending=True)
    .to_pandas()
    .style.format(precision=0)
    .background_gradient(vmax=35_000)
    # .to_markdown()
)

Performance count by sex and event, colored:


,Event,female,male
0,Hammer Throw,33647,12984
1,100 Metres,26983,24875
2,200 Metres,22067,15005
3,Pole Vault,15670,16388
4,3000 Metres Steeplechase,15125,9665
5,Javelin Throw,13449,7564
6,800 Metres,11597,7283
7,400 Metres,10867,8189
8,1500 Metres,9456,9107
9,20 Kilometres Race Walk,9275,3773


In [126]:
import altair as alt
from camminapy.plot import altair_theme

print("Count of performances grouped by year (starting 1960):")

altair_theme()
alt.Chart(
    df.with_columns(pl.col("Date").dt.year())
    .groupby("Date", "Sex")
    .count()
    .filter(pl.col("Date") > 1960)
    .sort("Date")
    .to_pandas()
).mark_bar(clip=True).encode(
    x=alt.X("Date:N").axis(labelAngle=-90, values=list(range(1960, 2024, 2))),
    y="count:Q",
    color=alt.Color("Sex:N").scale(
        domain=["female", "male"], range=["purple", "green"]
    ),
).properties(
    height=300, width=550
)

Count of performances grouped by year (starting 1960):


alt.Chart(...)

Interesting to see COVID pop up in this data as well.

In [127]:
print("The top 10 events with the most performances:")
(
    df.groupby("Event")
    .count()
    .sort("count", descending=True)
    .head(10)
    .to_pandas()
    .style.background_gradient(subset="count")
)

The top 10 events with the most performances:


,Event,count
0,100 Metres,51858
1,Hammer Throw,46631
2,200 Metres,37072
3,Pole Vault,32058
4,3000 Metres Steeplechase,24790
5,Shot Put,23407
6,400 Metres Hurdles,22480
7,Javelin Throw,21013
8,400 Metres,19056
9,800 Metres,18880


In [137]:
print(
    "Let's finish with an overview that shows during which months, which events are held:"
)
alt.Chart(
    df.with_columns(pl.col("Date").dt.month())
    .groupby("Date", "Sex", "Event")
    .count()
    .filter(pl.col("Event").str.contains("Walk").is_not())
    .sort("Date")
    .to_pandas()
).mark_bar(clip=True).encode(
    x=alt.X("Date:N").title("Month of Year"),
    y="count:Q",
    color=alt.Color("Sex:N").scale(
        domain=["female", "male"], range=["purple", "green"]
    ),
    row="Event:N",
).properties(
    height=200, width=550
).resolve_scale(
    y="independent"
)

alt.Chart(...)